<a href="https://colab.research.google.com/github/Hrishkesh/Invoice-/blob/master/Invoice_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!ls drive/My\ Drive\/invoice_eng

'Colab Notebooks'	       outdir_np_bbox_anchor_mask   output
 network.py		       outdir_np_chargrid_1h
 outdir_np_bbox_anchor_coord   outdir_np_gt_1h


In [0]:
!pip install --user tensorflow==2.0.0

In [0]:
!pip install --upgrade wandb
!wandb login 47ae077cee2c514d2b004ca904e0a365185a06c1

     |████████████████████████████████| 1.4MB 4.9MB/s 
     |████████████████████████████████| 112kB 35.5MB/s 
     |████████████████████████████████| 102kB 8.5MB/s 
     |████████████████████████████████| 102kB 9.9MB/s 
     |████████████████████████████████| 460kB 31.0MB/s 
     |████████████████████████████████| 71kB 6.4MB/s 
     |████████████████████████████████| 71kB 6.9MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=29da869662401c0ed5fc3fda7d61a06286d75e468a2acae418a81d0ece5b6d64
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for watchdog: filename=watchdog-0.10.2-cp36-none-any.whl size=73605 sha256=5b1fd48183d21305df9ab99db79c49d5eda088800002856ea768969a45c1f3f8
  Stored in directory: /root/.cache/pip/wheels/bc/ed/6c/028dea90d31b359cd2a7c8b0da4db80e41d24a59614154072e
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=bb47

In [0]:
!python drive/My\ Drive\/invoice_eng/network.py

2020-05-23 13:26:15.399446: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-05-23 13:26:15.404052: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2020-05-23 13:26:15.404288: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2512bc0 executing computations on platform Host. Devices:
2020-05-23 13:26:15.404325: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4973467035375093879
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 14248158465058440542
physical_device_desc: "device: XLA_CPU device"
]
Tensorflow version:  2.0.0
Test GPU:  
trainset:  495  - testset:  123
Train on 6 samples
6/1 [====================================================

In [0]:
import tensorflow as tf
tf.__version__

'2.0.0'